In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import string

In [20]:
df = pd.read_csv('finalSentimentdata2.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,sentiment,text
0,sad,agree the poor in india are treated badly thei...
1,joy,if only i could have spent the with this cutie...
2,joy,will nature conservation remain a priority in ...
3,sad,coronavirus disappearing in italy show this to...
4,sad,uk records lowest daily virus death toll since...
...,...,...
3085,sad,today at 02 30pm a 54 year old bangladeshi mal...
3086,anger,corona virus i implore that you cease activity...
3087,joy,issa date once lockdown ends inshaallah (and c...
3088,sad,the death toll due to covid 19 rose to 31 in j...


In [21]:
df.duplicated().sum()

42

In [22]:
df.drop_duplicates(inplace=True)

In [23]:
num_values = df['sentiment'].value_counts()
num_values

sentiment
fear     796
sad      770
anger    766
joy      716
Name: count, dtype: int64

In [24]:
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text
    

def clean_text_column(df, text_col_name):
    """Clean the text column by removing rows with empty text while retaining specific symbols (!, ?, .)."""
    # Drop rows with missing or empty text
    start_len = len(df)
    df = df[df[text_col_name].notnull()]  # Remove NaN values
    df = df[df[text_col_name].str.strip().astype(bool)]  # Remove empty strings

    # Clean text by removing unusual symbols, non english text, except !, ?, .
    def clean_text(text):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$", '', text, flags=re.MULTILINE)  # Remove URLs
        text = re.sub(r'[^a-zA-Z0-9\s!?.,]', '', text)  # Remove non-alphanumeric characters except !, ?, .
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        return text.strip()  # Strip leading/trailing spaces

    df[text_col_name] = df[text_col_name].apply(clean_text)

    # Define a function to filter out rows with insufficient content
    def has_valid_content(text):
        # Remove rows that only contain symbols like !, ?, or .
        if re.fullmatch(r'[!?.,\s]*', text):
            return False
        # Removes rows with less than 4 chars
        if len(text.strip()) < 4:
            return False
        return True

    # Apply the filter
    df = df[df[text_col_name].apply(has_valid_content)]
    cur_len = len(df)
    print(f"Clean dataset's text. started with {start_len} rows, after cleaning: {cur_len}")
    return df

text_col_name = 'text'

# Remove HTML tags
df[text_col_name] = df[text_col_name].apply(remove_tags)

# Clean text column
print(f'text column name: {text_col_name}')
df = clean_text_column(df, text_col_name)

# Disabled stopwords removal
# sw_list = stopwords.words('english')
# df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

text column name: text
Clean dataset's text. started with 3048 rows, after cleaning: 3046


In [26]:
label_mapping = {'sad': 0, 'joy': 1, 'fear': 2, 'anger': 3}
df['sentiment_label'] = df['sentiment']
df['sentiment'] = df['sentiment'].map(label_mapping)

In [27]:
df

,sentiment,text,sentiment_label
0,0,agree the poor in india are treated badly thei...,sad
1,1,if only i could have spent the with this cutie...,joy
2,1,will nature conservation remain a priority in ...,joy
3,0,coronavirus disappearing in italy show this to...,sad
4,0,uk records lowest daily virus death toll since...,sad
...,...,...,...
3084,3,grumpy or dopey who wins the tonight by scarin...,anger
3086,3,corona virus i implore that you cease activity...,anger
3087,1,issa date once lockdown ends inshaallah and co...,joy
3088,0,the death toll due to covid 19 rose to 31 in j...,sad


In [28]:
df.to_csv("preproccessed_finalSentimentdata2.csv", index=False)